In [1]:
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

In [2]:
%matplotlib inline

from pathlib import Path
import numpy as np
import pandas as pd
import pandas_datareader.data as web

from pathlib import Path
import numpy as np
import pandas as pd
import gc

import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
from sklearn.preprocessing import scale
import talib

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
DATA_STORE = Path('/home/sayem/Desktop/Project/data/assets.h5')

with pd.HDFStore(DATA_STORE) as store:
    data = store['data/top500_dataset']

In [4]:
selected_data = data[['open', 'high', 'low', 'close', 'volume']]

In [5]:
selected_data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1336327 entries, ('AA', Timestamp('2013-01-03 00:00:00')) to ('ZION', Timestamp('2023-08-10 00:00:00'))
Data columns (total 5 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   open    1336327 non-null  float64
 1   high    1336327 non-null  float64
 2   low     1336327 non-null  float64
 3   close   1336327 non-null  float64
 4   volume  1336327 non-null  float64
dtypes: float64(5)
memory usage: 56.5+ MB


In [6]:
# import numpy as np
# import matplotlib.pyplot as plt
# from tsfracdiff import FractionalDifferentiator

# def process_tickers(df):
#     all_orders = []

#     for ticker, group in df.groupby(level=0):
#         group = group.reset_index(drop=True)  # Reset the multi-index for the current group

#         # Focus on the 'close' prices
#         X_ticker = group[['close']]
#         # print(X_ticker)
#         # break

#         # Calculate percentage change (returns)
#         X_pct_change = X_ticker.pct_change().dropna()

#         # 1. Fractional Differentiation for the specific ticker
#         fracDiff = FractionalDifferentiator()
#         X_frac = fracDiff.FitTransform(X_ticker)

#         # Print estimated orders for the specific ticker
#         orders = fracDiff.orders
#         print(f"For ticker {ticker}, estimated orders range from {min(orders)} to {max(orders)}.")
#         all_orders.extend(orders)

#         # 2. First Difference
#         X_diff = X_ticker.diff().dropna()

#         # 3. Plotting
#         fig, axes = plt.subplots(3, 1, figsize=(15, 12))
        
#         X_diff.plot(title=f'{ticker} - Differenced Prices', ax=axes[0], color='blue')
#         axes[0].set_ylabel('Differenced Close')
        
#         X_ticker.plot(ax=axes[1], label='Original Close', color='blue')
#         X_frac.plot(title=f'{ticker} - Fractionally Differentiated Prices vs Original Close', ax=axes[1], linestyle='--', color='orange')
#         axes[1].set_ylabel('Close Prices')
#         axes[1].legend()
        
#         X_pct_change.plot(title=f'{ticker} - Percentage Change (Returns)', ax=axes[2], color='green')
#         axes[2].set_ylabel('Percentage Change')

#         plt.tight_layout()
#         plt.show()

#         # 4. Plotting Correlations
#         fig, ax = plt.subplots(figsize=(12, 6))
#         correlations = {
#             'Differenced Close': X_diff.corrwith(X_ticker)['close'],
#             'Fractional Difference': X_frac.corrwith(X_ticker)['close'],
#             'Percentage Change': X_pct_change.corrwith(X_ticker).iloc[0]
#         }
        
#         ax.bar(correlations.keys(), correlations.values(), alpha=0.7)
#         ax.set_title(f'{ticker} - Correlation of Transformed Features with Original')
#         ax.set_ylabel('Correlation Coefficient')
#         plt.tight_layout()
#         plt.show()
#         # break

#     # Analyzing orders after processing all tickers
#     print(f"Across all tickers, estimated orders range from {min(all_orders)} to {max(all_orders)}.")

# # Assuming df is your DataFrame
# process_tickers(selected_data)

In [7]:
# import numpy as np
# import matplotlib.pyplot as plt
# import pandas as pd
# from tsfracdiff import FractionalDifferentiator

# def process_tickers(df):
#     processed_dfs = []  # Container to store modified dataframes
#     i = 0
#     for ticker, group in df.groupby(level=0):
#         group = group.copy().reset_index(drop=True)  # Reset the multi-index for the current group

#         # Calculate X_frac (I'm assuming you have a method to do this)
#         X_frac = FractionalDifferentiator().FitTransform(group[['close']])

#         # The number of NaN values needed to align X_frac with close
#         nans_needed = len(group) - len(X_frac)

#         # Create a NaN series of required length
#         nan_series = pd.Series([np.nan] * nans_needed)

#         # Append the NaN series to the start of X_frac
#         X_frac_aligned = nan_series.append(X_frac['close']).reset_index(drop=True)

#         # Add the aligned X_frac to the group
#         group['X_frac'] = X_frac_aligned

#         processed_dfs.append(group)
#         if (i > 5):
#             break
#         i += 1

#     # Concatenate all processed dataframes
#     result_df = pd.concat(processed_dfs)

#     return result_df

# updated = process_tickers(selected_data)

In [8]:
import numpy as np
import pandas as pd
from tsfracdiff import FractionalDifferentiator

def process_tickers(df, num_tickers=None):
    processed_dfs = []

    # If num_tickers is not provided, process all tickers
    if num_tickers is None:
        ticker_groups = list(df.groupby(level=0))
    else:
        ticker_groups = list(df.groupby(level=0))[:num_tickers]

    for ticker, group in ticker_groups:
        group_copy = group.copy()

        # Calculate X_frac 
        frac_diff = FractionalDifferentiator()
        X_frac = frac_diff.FitTransform(group_copy[['close']])
    
        # The number of NaN values needed to align X_frac with close
        nans_needed = len(group_copy) - len(X_frac)

        # Create a NaN series of required length
        nan_series = pd.Series([np.nan] * nans_needed)

        # Append the NaN series to the start of X_frac
        X_frac_aligned = nan_series.append(X_frac['close']).reset_index(drop=True)

        # Add the aligned X_frac to the group
        column_name = f"ret_frac_order"
        group_copy[column_name] = X_frac_aligned.values

        processed_dfs.append(group_copy)

    # Concatenate all processed dataframes
    result_df = pd.concat(processed_dfs)

    return result_df

# Process all tickers
updated = process_tickers(selected_data)

In [9]:
from utils import  optimize_dataframe
op = optimize_dataframe(updated)

Data memory before optimization: 66.40 MB
Data memory after optimization: 40.91 MB
Reduced by: 38.39%


In [10]:
op.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1336327 entries, ('AA', Timestamp('2013-01-03 00:00:00')) to ('ZTS', Timestamp('2023-08-10 00:00:00'))
Data columns (total 6 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   open            1336327 non-null  float32
 1   high            1336327 non-null  float32
 2   low             1336327 non-null  float32
 3   close           1336327 non-null  float32
 4   volume          1336327 non-null  float64
 5   ret_frac_order  1336327 non-null  float32
dtypes: float32(5), float64(1)
memory usage: 40.9+ MB


In [11]:
# # ### Parallel code
# import numpy as np
# import pandas as pd
# from tsfracdiff import FractionalDifferentiator
# from concurrent.futures import ProcessPoolExecutor

# def process_one_ticker(data_group):
#     ticker, group = data_group
#     group_copy = group.copy()

#     # Calculate X_frac 
#     frac_diff = FractionalDifferentiator()
#     X_frac = frac_diff.FitTransform(group_copy[['close']])
    
#     # The number of NaN values needed to align X_frac with close
#     nans_needed = len(group_copy) - len(X_frac)

#     # Create a NaN series of required length
#     nan_series = pd.Series([np.nan] * nans_needed)

#     # Append the NaN series to the start of X_frac
#     X_frac_aligned = nan_series.append(X_frac['close']).reset_index(drop=True)

#     # Add the aligned X_frac to the group
#     column_name = f"ret_frac_order"
#     group_copy[column_name] = X_frac_aligned.values

#     return group_copy

# def process_tickers_parallel(df, num_tickers=None, max_workers=20):
#     # If num_tickers is not provided, process all tickers
#     if num_tickers is None:
#         ticker_groups = list(df.groupby(level=0))
#     else:
#         ticker_groups = list(df.groupby(level=0))[:num_tickers]

#     with ProcessPoolExecutor(max_workers=max_workers) as executor:
#         processed_dfs = list(executor.map(process_one_ticker, ticker_groups))

#     # Concatenate all processed dataframes
#     result_df = pd.concat(processed_dfs)

#     return result_df

# # Usage:
# updated = process_tickers_parallel(selected_data)

In [12]:
updated

open        high         low       close  \
ticker date                                                         
AA     2013-01-03   18.904100   19.306499   18.883200   19.137600   
       2013-01-04   19.222000   19.579800   19.200300   19.537100   
       2013-01-07   19.537100   19.600599   19.200300   19.200300   
       2013-01-08   19.327400   19.389099   19.158400   19.200300   
       2013-01-09   19.685101   19.685101   19.115700   19.158400   
...                       ...         ...         ...         ...   
ZTS    2023-08-04  180.139999  182.460007  178.339996  178.630005   
       2023-08-07  180.190002  181.880005  180.029999  180.690002   
       2023-08-08  182.149994  190.539993  177.250000  189.300003   
       2023-08-09  189.940002  190.949997  186.964996  189.350006   
       2023-08-10  190.000000  191.824997  187.729996  189.100006   

                         volume  ret_frac_order  
ticker date                                      
AA     2013-01-03  1.057118e+07        8.914233  
       2013-01-04  8.690739e+06        8.914233  
       2013-01-07  1.031992e+07        8.914233  
       2013-01-08  1.609879e+07        8.914233  
       2013-01-09  1.979932e+07        8.914233  
...                         ...             ...  
ZTS    2023-08-04  1.827463e+06       13.458624  
       2023-08-07  1.941699e+06       16.669207  
       2023-08-08  4.117986e+06       24.922548  
       2023-08-09  2.270030e+06       21.726665  
       2023-08-10  2.348639e+06       20.455498  

[1336327 rows x 6 columns]